In [1]:
import pandas as pd
import numpy as np
from ortools.sat.python import cp_model

In [2]:
data = pd.read_csv('ProbMatrix.csv')

In [3]:
names = data['Team']
teams = range(32)
weeks = range(18)
probs = data.drop('Team', axis=1).to_numpy()

In [4]:
model = cp_model.CpModel()

# Generate all the binary decision variables
x = {}
for t in teams:
    for w in weeks:
        x[(t, w)] = model.NewBoolVar(f'x{t}{w}')

# Team Constraint
for t in teams:
    model.Add(sum(x[(t, w)] for w in weeks) <= 1)

# Week Constraint
for w in weeks:
    model.Add(sum(x[(t, w)] for t in teams) == 1)

# Objective function
model.Maximize(
    sum(x[(t, w)]*np.log(probs[t][w] + 1e-6) for t in teams for w in weeks)
)

In [5]:
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 10
solver.parameters.num_search_workers = 4
status = solver.Solve(model)

In [6]:
# Print the winning lineup
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    res = 'Optimal' if status == cp_model.OPTIMAL else 'Feasible'
    print('{} Solution'.format(res))
    final_prob = 0
    for w in weeks:
        for t in teams:
            if solver.Value(x[(t, w)]) == 1:
                print(f'Week {1 + w}: {names.iloc[t]} ({round(100*probs[t][w], 1)}%)')
                final_prob += np.log(probs[t][w])
    print(f'1 in {int(1 / np.exp(final_prob))} chance of winning')

Optimal Solution
Week 1: Seattle Seahawks (69.0%)
Week 2: Green Bay Packers (64.4%)
Week 3: New York Jets (73.9%)
Week 4: San Francisco 49ers (82.3%)
Week 5: Chicago Bears (67.0%)
Week 6: Houston Texans (66.6%)
Week 7: Jacksonville Jaguars (70.7%)
Week 8: Detroit Lions (74.9%)
Week 9: Baltimore Ravens (81.3%)
Week 10: Kansas City Chiefs (78.3%)
Week 11: Los Angeles Rams (64.8%)
Week 12: Miami Dolphins (75.2%)
Week 13: Dallas Cowboys (77.1%)
Week 14: Philadelphia Eagles (76.2%)
Week 15: Arizona Cardinals (63.3%)
Week 16: Buffalo Bills (77.4%)
Week 17: Cincinnati Bengals (77.0%)
Week 18: Atlanta Falcons (70.5%)
1 in 323 chance of winning
